In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# --------------------------------------------------------------
# Load outputs from previous team members
# --------------------------------------------------------------
mean_face = np.load("mean_face.npy")
eigenfaces = np.load("eigenfaces.npy")
eigenvalues = np.load("eigenvalues.npy")

# Load original training data
df = pd.read_csv("X_TRAIN.csv")
X_train = df.values
X_train = np.nan_to_num(X_train)

print("Loaded shapes:")
print("X_train:", X_train.shape)
print("mean_face:", mean_face.shape)
print("eigenfaces:", eigenfaces.shape)
print("eigenvalues:", eigenvalues.shape)

# --------------------------------------------------------------
# FIX: Re-normalize eigenfaces to ensure they are orthonormal
# --------------------------------------------------------------
# Previous code used np.linalg.norm(..., axis=0) which normalizes to unit norm
# But due to numerical precision in large datasets, it may not be perfect
# We re-normalize here to fix any issues causing perfect reconstruction on training faces
norms = np.linalg.norm(eigenfaces, axis=0)
# Avoid division by zero (though unlikely)
norms[norms == 0] = 1.0
eigenfaces = eigenfaces / norms

print("\nEigenfaces re-normalized for orthonormality.")

# --------------------------------------------------------------
# AUTOMATICALLY SELECT THE MOST DISTINCTIVE FACE
# --------------------------------------------------------------
centered_data = X_train - mean_face
distances_from_mean = np.linalg.norm(centered_data, axis=1)

sample_idx = np.argmax(distances_from_mean)
max_distance = distances_from_mean[sample_idx]

print(f"\nSelected most distinctive face: index {sample_idx}")
print(f"Distance from mean face: {max_distance:.2f}")

if max_distance < 100:
    print("WARNING: Low variation in dataset — results may still show low MSE.")
else:
    print("Good variation — expect clear reconstruction improvement with higher k!")

original_face = X_train[sample_idx].reshape(-1)

# Add noise
noise_level = 75
noisy_face = original_face + np.random.normal(0, noise_level, original_face.shape)
noisy_face = np.clip(noisy_face, 0, 255)

clean_centered = original_face - mean_face
noisy_centered = noisy_face - mean_face

# --------------------------------------------------------------
# Helper functions (unchanged)
# --------------------------------------------------------------
def project_to_eigenspace(centered_face, eigenfaces, k):
    return np.dot(eigenfaces[:, :k].T, centered_face)

def reconstruct_from_projection(coeffs, eigenfaces, k, mean_face):
    recon_centered = np.dot(eigenfaces[:, :k], coeffs)
    return recon_centered + mean_face

# --------------------------------------------------------------
# Test components
# --------------------------------------------------------------
component_sizes = [10, 25, 50, 100]
mse_clean = {}
mse_noisy = {}

num_plots = len(component_sizes) + 2
fig, axes = plt.subplots(2, num_plots, figsize=(3 * num_plots, 6))
axes = axes.flatten()

axes[0].imshow(original_face.reshape(100, 100), cmap='gray')
axes[0].set_title("Original Clean")
axes[0].axis('off')

axes[1].imshow(noisy_face.reshape(100, 100), cmap='gray')
axes[1].set_title(f"Noisy (σ={noise_level})")
axes[1].axis('off')

plot_idx = 2
for k in component_sizes:
    # Clean
    coeffs_clean = project_to_eigenspace(clean_centered, eigenfaces, k)
    recon_clean = reconstruct_from_projection(coeffs_clean, eigenfaces, k, mean_face)
    recon_clean = np.clip(recon_clean, 0, 255)
    mse_clean[k] = np.mean((recon_clean - original_face) ** 2)

    # Denoised
    coeffs_noisy = project_to_eigenspace(noisy_centered, eigenfaces, k)
    recon_noisy = reconstruct_from_projection(coeffs_noisy, eigenfaces, k, mean_face)
    recon_noisy = np.clip(recon_noisy, 0, 255)
    mse_noisy[k] = np.mean((recon_noisy - original_face) ** 2)

    axes[plot_idx].imshow(recon_noisy.reshape(100, 100), cmap='gray')
    axes[plot_idx].set_title(f"Recon (k={k})\nMSE={mse_noisy[k]:.1f}")
    axes[plot_idx].axis('off')

    print(f"k = {k:3d} | Clean recon MSE: {mse_clean[k]:8.1f} | Denoised MSE: {mse_noisy[k]:8.1f}")

    plot_idx += 1

plt.suptitle("PCA-based Image Denoising: Original → Noisy → Reconstructions with varying k", fontsize=14)
plt.tight_layout()
plt.show()
